<a href="https://colab.research.google.com/github/Pablothewall/NLP_MINUTES_FED/blob/main/Hugging_Face_FOMC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig

In [3]:
tokenizer = AutoTokenizer.from_pretrained("gtfintechlab/FOMC-RoBERTa", do_lower_case=True, do_basic_tokenize=True)
model = AutoModelForSequenceClassification.from_pretrained("gtfintechlab/FOMC-RoBERTa", num_labels=3)
config = AutoConfig.from_pretrained("gtfintechlab/FOMC-RoBERTa")

classifier = pipeline('text-classification', model=model, tokenizer=tokenizer, config=config, device=0, framework="pt",
    return_all_scores=True)
label_mapping = {
    'LABEL_0': 'Dovish',
    'LABEL_1': 'Hawkish',
    'LABEL_2': 'Neutral'
}

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/891 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
path='/content/drive/MyDrive/minutes.pkl'

In [5]:
import pandas as pd
df=pd.read_pickle(path)

In [6]:
df.reset_index(inplace=True)
df=df[['Date', 'Minutes']]
df

,Date,Minutes
0,1995-02-01,A meeting of the Federal Open Market Committee...
1,1995-03-28,A meeting of the Federal Open Market Committee...
2,1995-05-23,A meeting of the Federal Open Market Committee...
3,1995-07-06,A meeting of the Federal Open Market Committee...
4,1995-08-22,A meeting of the Federal Open Market Committee...
...,...,...
201,2023-05-03,"The Federal Reserve, the central bank of the U..."
202,2023-06-14,"The Federal Reserve, the central bank of the U..."
203,2023-07-26,"The Federal Reserve, the central bank of the U..."
204,2023-09-20,"The Federal Reserve, the central bank of the U..."


In [7]:
label_mapping = {
    'LABEL_0': 'Dovish',
    'LABEL_1': 'Hawkish',
    'LABEL_2': 'Neutral'
}

In [8]:
df.columns = ['Date', 'Federal_Reserve_Mins']

In [9]:
import pandas as pd

# Resetting the index and renaming columns



# Splitting the sentences in 'Federal_Reserve_Mins' column
df['Federal_Reserve_Mins'] = df['Federal_Reserve_Mins'].apply(lambda x: x.lower().split('. '))

# Creating a new DataFrame to store the results
new_rows = []

# Iterating through the rows of the original DataFrame
for index, row in df.iterrows():
    date = row['Date']
    sentences = row['Federal_Reserve_Mins']

    # Concatenating sentences until the total length exceeds 700 characters
    current_chunk = ""
    for sentence in sentences:
        if len(current_chunk) + len(sentence) <= 700:
            current_chunk += sentence
        else:
            new_rows.append({'Date': date, 'Federal_Reserve_Mins': current_chunk})
            current_chunk = sentence

    # Adding the last chunk if it exists
    if current_chunk:
        new_rows.append({'Date': date, 'Federal_Reserve_Mins': current_chunk})

# Creating a new DataFrame from the list of new rows
sentences_split = pd.DataFrame(new_rows)




In [10]:
sentences_split

,Date,Federal_Reserve_Mins
0,1995-02-01,a meeting of the federal open market committee...
1,1995-02-01,"broaddus, forrestal, and parry, presidents ..."
2,1995-02-01,"madigan, associate director, division of ..."
3,1995-02-01,"o'day,1 associate general counsel, legal ..."
4,1995-02-01,"tschinkel, senior vice presidents, fede..."
...,...,...
96243,2023-11-01,in determining the extent of additional policy...
96244,2023-11-01,in assessing the appropriate stance of moneta...
96245,2023-11-01,"jefferson, neel kashkari, adriana dkugler, lor..."
96246,2023-11-01,the meeting adjourned at 10:05 a.mon november ...


In [1]:
sentences_split_1=sentences_split[:3000]
sentences_split_1

NameError: name 'sentences_split' is not defined

In [44]:
label_mapping = {
    'LABEL_0': 'Dovish',
    'LABEL_1': 'Hawkish',
    'LABEL_2': 'Neutral'
}
def translate_result(result_i):
  return pd.Series({
      label_mapping[dict_i["label"]] : dict_i["score"]
      for dict_i in result_i[0]
  })

In [51]:
import numpy as np
def classifier_sentences(text):
  try:
    return translate_result(classifier([text],batch_size=128))
  except:
    return pd.Series({
    'Dovish':np.nan,
    'Hawkish':np.nan,
    'Neutral':np.nan
})

In [56]:
%%time
result=classifier(sentences_split_1['Federal_Reserve_Mins'].tolist(), batch_size=128)


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


RuntimeError: The expanded size of the tensor (543) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [128, 543].  Tensor sizes: [1, 514]

In [53]:
sentences_split_1.loc[:, ['Dovish', 'Hawkish', 'Neutral']]=sentences_split_1['Federal_Reserve_Mins'].apply(classifier_sentences)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentiall

In [54]:
sentences_split_1

,Date,Federal_Reserve_Mins,Dovish,Hawkish,Neutral
750,1996-01-30,the occasional periods of firmness in reser...,0.731814,0.047372,0.220815
751,1996-01-30,both were seen as suggesting slower econom...,0.997726,0.001869,0.000404
752,1996-01-30,the dollar's upward movement against the g...,0.409327,0.017227,0.573446
753,1996-01-30,the pickup in m2 growth partly reflected th...,0.305347,0.004468,0.690185
754,1996-01-30,faster growth of m3 in december and january...,0.018419,0.092057,0.889524
...,...,...,...,...,...
1495,1996-01-30,the dollar's upward movement against the g...,0.409327,0.017227,0.573446
1496,1996-01-30,the pickup in m2 growth partly reflected th...,0.305347,0.004468,0.690185
1497,1996-01-30,faster growth of m3 in december and january...,0.018419,0.092057,0.889524
1498,1996-01-30,the staff forecast prepared for this mee...,0.002356,0.000120,0.997524


In [38]:
sentences_split_1.loc[:, ['Dovish', 'Hawkish', 'Neutral']] = .apply(translate_result)


<ipython-input-38-987c5ca74828>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sentences_split_1.loc[:, ['Dovish', 'Hawkish', 'Neutral']] = sentences_split_1['Prediction'].apply(translate_result)


In [39]:
sentences_split_1.groupby("Date")[["Dovish",	"Hawkish",	"Neutral"]].mean()


,Dovish,Hawkish,Neutral
Date,,,
1999-05-18,0.045285,0.340914,0.613801
1999-06-29,0.057703,0.289087,0.653209
1999-08-24,0.039840,0.303996,0.656164
1999-10-05,0.019823,0.362296,0.617881
1999-11-16,0.043047,0.383558,0.573395
...,...,...,...
2023-05-03,0.088228,0.227837,0.683935
2023-06-14,0.067132,0.262928,0.669939
2023-07-26,0.112014,0.348600,0.539386


In [ ]:
sentences_split_1 = pd.DataFrame()
chunk_size=1
# Iterate through the chunks
for i in range(0, 3750, chunk_size):

    # Select a chunk of 500 rows


    chunk = sentences_split.iloc[i:i + chunk_size]
    processed_chunk=sentences_split.iloc[i:i + chunk_size]
    # Apply your function to the chunk
    processed_chunk["Prediction"] = chunk['Federal_Reserve_Mins'].apply(lambda x: classifier([x],batch_size=128))  # Replace 'your_function' with your actual function

    # Concatenate the processed chunk to the result DataFrame
    sentences_split_1 = pd.concat([sentences_split_1, processed_chunk])
    print(len(sentences_split_1))